# Keras und MNIST

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import random

%matplotlib inline
%config InlineBackend.figure_format = 'retina'

In [ ]:
!python --version
print("NumPy: " + np.__version__)
print("Keras: " + tf.keras.__version__)
print("TensorFlow: " + tf.__version__)

# Hardwareunterstützung?
tf.config.list_physical_devices()

### Datensatz importieren

In [ ]:
(train_images, train_labels), (test_images, test_labels) = tf.keras.datasets.mnist.load_data()

In [ ]:
# Daten anzeigen
# Anzeige der Größe/Dimensionen der Variablen

print("train_images shape", train_images.shape)
print("train_labels shape", train_labels.shape)
print("test_images shape", test_images.shape)
print("test_labels shape", test_labels.shape)

In [ ]:
# Zufällige Auswahl an Bildern aus dem Trainingsdatensatz anzeigen

plt.rcParams['figure.figsize'] = (16,7)

for i in range(18):
    plt.subplot(3,6,i+1)
    num = random.randint(0, len(train_images))
    plt.imshow(train_images[num], cmap='gray', interpolation='none')
    plt.title("Label: {}".format(train_labels[num]))
plt.tight_layout()

#### Serialisieren

In [ ]:
# Umformen des Trainings- und Testdatensatzes
# 28x28 Pixel => Vektor mit 784 Pixeln
train_images = train_images.reshape(60000, 784)
test_images  = test_images.reshape(10000, 784)

#### Normalisieren

In [ ]:
# Konvertierung Integer => 32-Bit Fließkommawert
train_images = train_images.astype('float32')
test_images  = test_images.astype('float32')

# Normalisieren der Werte
# Jedes Pixel besitzt einen Wert zwischen 0...1
train_images = train_images / 255
test_images  = test_images / 255

print("Trainingsdaten:", train_images.shape)
print("Testdaten:", test_images.shape)

#### Ausgangswerte ins 1-Hot-Format umformen

Umformen der Klassen (eindeutige Ziffern) ins One-Hot-Format:

```
0 -> [1, 0, 0, 0, 0, 0, 0, 0, 0]
1 -> [0, 1, 0, 0, 0, 0, 0, 0, 0]
2 -> [0, 0, 1, 0, 0, 0, 0, 0, 0]
...
9 -> [0, 0, 0, 0, 0, 0, 0, 0, 1]
```

In [ ]:
# Labels original
train_labels

In [ ]:
nb_classes = 10 # Anzahl an eindeutigen Ziffern

train_labels = tf.keras.utils.to_categorical(train_labels, nb_classes)
test_labels  = tf.keras.utils.to_categorical(test_labels, nb_classes)

### Entwurf des neuronalen Netzes

In [ ]:
model = tf.keras.Sequential()
model.add(tf.keras.layers.Input(shape=[784,]))
model.add(tf.keras.layers.Dense(128, activation='relu'))
model.add(tf.keras.layers.Dense(64, activation='relu'))
model.add(tf.keras.layers.Dense(10, activation='softmax'))

# Zusammenfassung
model.summary()

In [ ]:
# Kompilierung
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

### Training des neuronalen Netzes

In [ ]:
model.fit(train_images, train_labels, batch_size=200, epochs=20, verbose=1)

### Evaluation

In [ ]:
scores = model.evaluate(test_images, test_labels, verbose=1)
print("Fehler: %.2f%%" % (100-scores[1]*100))

### Vorhersage

In [ ]:
index = 3   # => Indexwert des Testdatensatzes

predict = model.predict(test_images)
#print("Vorhersage: %d" % predict[index].argmax())
#print("Label:      %d" % test_labels[index].argmax())

# Grafische Ausgabe des Ergebnisses
plt.imshow(test_images[index].reshape(28,28), cmap=plt.get_cmap('gray'));
plt.title("Vorhersage: %d, Label: %d" % (predict[index].argmax(), test_labels[index].argmax()));

### Performance des Netzes

In diesem Abschnitt untersuchen wir, welche Bilder richtig zugeordnet wurden und bei welchen Bildern dies nicht gelingt. Dazu stellt man typischerweise eine sog. Konfusionsmatrix auf. Die Konfusionsmatrix zeigt, welche Klassen erkannt wurden und wie der wahre Wert lautet.

In [ ]:
import sklearn
import itertools

print("sklearn: " + sklearn.__version__)

In [ ]:
def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Konfusionsmatrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    plt.figure(figsize=(6,6))
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    #plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]

    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, cm[i, j],
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('Wahre Klasse')
    plt.xlabel('Vorhergesagte Klasse')

In [ ]:
confusion_mtx = sklearn.metrics.confusion_matrix(test_labels.argmax(axis=1), predict.round().argmax(axis=1))
plot_confusion_matrix(confusion_mtx, classes = range(10))

### Fehleranalyse

In [ ]:
# Finde die Indizes, bei denen die vorhergesagten Werte nicht mit den Labels übereinstimmen
np.where(np.not_equal(np.argmax(predict, axis=1), np.argmax(test_labels, axis=1)))

In [ ]:
index = 18

label = ('0', '1', '2', '3', '4', '5', '6', '7', '8', '9')

f = plt.figure(figsize=(10,4))
ax1 = f.add_subplot(121)
ax2 = f.add_subplot(122)

ax1.bar(label, predict[index], align='center')
ax1.set_ylabel('Wahrscheinlichkeit')
ax1.set_title("Vorhersage: %d, Label: %d" % (predict[index].argmax(), test_labels[index].argmax()));

ax2.imshow(test_images[index].reshape(28,28), cmap=plt.get_cmap('gray'));

## Portierung auf Mikrocontroller

#### Hilfsfunktionen

- Funktion `get_file_size()` ermittelt die Dateigröße
- Funktion `convert_bytes()` konvertiert Bytes in KB und MB

In [ ]:
import os
import h5py
import sys

In [ ]:
def get_file_size(file_path):
  size = os.path.getsize(file_path)
  return size

def convert_bytes(size, unit=None):
  if unit == "KB":
    return print('Dateigröße: ' + str(round(size / 1024, 3)) + ' Kilobytes')
  elif unit == "MB":
    return print('Dateigröße: ' + str(round(size / (1024 * 1024), 3)) + ' Megabytes')
  else:
    return print('Dateigröße: ' + str(size) + ' bytes')

### Speichern des Keras-Modells

In [ ]:
# Keras-Modell speichern im H5-Format

KERAS_MODEL_FILENAME = 'tf_mnist.keras'
model.save(KERAS_MODEL_FILENAME)

In [ ]:
# Größe des Modells bestimmen
convert_bytes(get_file_size(KERAS_MODEL_FILENAME), "MB")

# Speichern der Modellgröße
KERAS_MODEL_SIZE = get_file_size(KERAS_MODEL_FILENAME)

### Konvertierung in TensorFlow lite-Modell **ohne** Quantisierung

In [ ]:
converter = tf.lite.TFLiteConverter.from_keras_model(model)

tflite_model = converter.convert()
open('tf_mnist.tflite', 'wb').write(tflite_model);

#### Analyse des TensorFlow lite-Modells

- Ausgabe: html-Seite (mit Browser öffnen)

In [ ]:
!python -m tensorflow.lite.tools.visualize tf_mnist.tflite visualized_model.html

... oder mittels

In [ ]:
tf.lite.experimental.Analyzer.analyze(model_content=tflite_model, gpu_compatibility=False)

#### Dateigröße des TensorFlow lite-Modells?

In [ ]:
convert_bytes(get_file_size('tf_mnist.tflite'), "KB")

### Konvertierung in TensorFlow lite-Modell **mit** Quantisierung

In [ ]:
converter = tf.lite.TFLiteConverter.from_keras_model(model)

converter.optimizations = [tf.lite.Optimize.DEFAULT]

tflite_model_quant = converter.convert()
open('tf_mnist_quantized.tflite', 'wb').write(tflite_model_quant);

#### Dateigröße des quantisierten TensorFlow lite-Modells?

In [ ]:
convert_bytes(get_file_size('tf_mnist_quantized.tflite'), "KB")

### Weitere Optimierung

In [ ]:
converter.representative_dataset = representative_dataset

converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]

converter.inference_input_type  = tf.uint8
converter.inference_output_type = tf.uint8

tflite_model_fully_quant = converter.convert()